In [2]:
from utils.data_utils import load_dataset, save_dataset
from utils.functions import load_problem
from utils import load_model

In [3]:
problem = load_problem('tsp')

In [4]:
reviser_path = 'pretrained_LCP/Reviser-6-FI/reviser_20/epoch-199.pt'
reviser, _ = load_model(reviser_path, is_local=True, mtl=False)
reviser.eval()
reviser.set_decode_type("greedy")

  [*] Loading model from pretrained_LCP/Reviser-6-FI/reviser_20/epoch-199.pt


In [11]:
dataset = reviser.problem.make_dataset(filename='data/FI_train_tsp/200_FI20_seed1235.pkl', num_samples=500000, offset=500000)

In [12]:
from torch.utils.data import DataLoader
dataloader = DataLoader(dataset, batch_size=500)

In [13]:
from tqdm import tqdm
import torch
from utils.functions import revision
import copy

original_subtour = torch.arange(20)

get_cost_func = lambda input, pi: problem.get_costs(input, pi, return_local=True)

# RG_dataset = []

for batch in tqdm(dataloader):
    decomposed_seeds = batch
    # a = copy.deepcopy(decomposed_seeds[4])
    decomposed_seeds = revision(get_cost_func, reviser, decomposed_seeds, original_subtour)
    # print(decomposed_seeds[4]==a)
    for instance in decomposed_seeds:
        RG_dataset.append(instance)
    # print(len(RG_dataset))


100%|██████████| 1000/1000 [04:44<00:00,  3.51it/s]


In [14]:
len(RG_dataset)

1000000

In [15]:
RG_dataset[0][0],RG_dataset[500000][0]

(tensor([1.0000, 0.2250]), tensor([0.0000, 0.6226]))

In [16]:
new = torch.stack(RG_dataset)

In [19]:
print(new.shape)

torch.Size([1000000, 20, 2])


In [18]:
torch.save(new, 'data/RG_train_tsp/RG20_seed1235.pt')

In [20]:
a = torch.load('data/RG_train_tsp/RG20_seed1235.pt')

In [21]:
a.shape

torch.Size([1000000, 20, 2])